# NYC Taxi Demand vs Used Car Price Analysis
## Hypothesis: Taxi demand correlates with used car prices in New York areas

This notebook analyzes the correlation between NYC taxi trip density (2019-2020) and Craigslist used car prices in NY regions using Spark for distributed processing.

## 1. Setup and Data Download

In [ ]:
# Install required packages
!pip install kaggle opendatasets pandas numpy matplotlib seaborn plotly -q

In [ ]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("TaxiCarPriceCorrelation") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .getOrCreate()

print(f"Spark version: {spark.version}")

### Download Datasets
**Note**: You need to set up Kaggle API credentials first:
1. Go to Kaggle → Account → Create New API Token
2. Place `kaggle.json` in `~/.kaggle/` directory
3. Run: `chmod 600 ~/.kaggle/kaggle.json`

In [ ]:
# Create data directory
os.makedirs('data', exist_ok=True)

# Download Craigslist cars dataset
print("Downloading Craigslist cars dataset...")
!kaggle datasets download -d austinreese/craigslist-carstrucks-data -p ./data --unzip

# Download NYC taxi dataset
print("\nDownloading NYC Yellow Taxi dataset...")
!kaggle datasets download -d microize/newyork-yellow-taxi-trip-data-2020-2019 -p ./data --unzip

print("\nDownload complete!")
!ls -lh data/

## 2. Data Loading and Exploration

In [ ]:
# Load Craigslist data
# Format is typically CSV
craigslist_df = spark.read.csv(
    "data/vehicles.csv",  # Adjust filename if different
    header=True,
    inferSchema=True
)

print("Craigslist Dataset Schema:")
craigslist_df.printSchema()
print(f"\nTotal records: {craigslist_df.count():,}")
print("\nSample data:")
craigslist_df.show(5, truncate=False)

In [ ]:
# Check available states in Craigslist data
print("States in dataset:")
craigslist_df.groupBy("state").count().orderBy(desc("count")).show(10)

In [ ]:
# Load NYC Taxi data
# Try both parquet and csv formats
taxi_files = !ls data/*.parquet data/*.csv 2>/dev/null || echo "no_files"

# Determine file format and load
taxi_path = "data/yellow_*.parquet" if any('.parquet' in f for f in taxi_files) else "data/*.csv"

if '.parquet' in taxi_path:
    taxi_df = spark.read.parquet(taxi_path)
else:
    taxi_df = spark.read.csv(taxi_path, header=True, inferSchema=True)

print("NYC Taxi Dataset Schema:")
taxi_df.printSchema()
print(f"\nTotal records: {taxi_df.count():,}")
print("\nSample data:")
taxi_df.show(5, truncate=False)

## 3. Data Preparation and Cleaning

### 3.1 Prepare Craigslist Data (Filter NY, Clean, Remove Outliers)

In [ ]:
# Filter for New York state only
ny_cars_raw = craigslist_df.filter(col("state") == "ny")

print(f"NY car listings: {ny_cars_raw.count():,}")

# Show date range
print("\nDate range in dataset:")
ny_cars_raw.select(
    min("posting_date").alias("min_date"),
    max("posting_date").alias("max_date")
).show()

In [ ]:
# Data cleaning and preparation
ny_cars_cleaned = ny_cars_raw \
    .filter(col("price").isNotNull()) \
    .filter(col("year").isNotNull()) \
    .filter(col("posting_date").isNotNull()) \
    .withColumn("posting_date", to_timestamp(col("posting_date"))) \
    .filter(year(col("posting_date")).isin([2019, 2020])) \
    .withColumn("price", col("price").cast("double")) \
    .withColumn("year", col("year").cast("int")) \
    .withColumn("odometer", col("odometer").cast("double"))

print(f"After filtering 2019-2020: {ny_cars_cleaned.count():,} records")

In [ ]:
# Calculate statistics for outlier detection
price_stats = ny_cars_cleaned.select(
    mean("price").alias("mean_price"),
    stddev("price").alias("std_price"),
    expr("percentile_approx(price, 0.01)").alias("p01"),
    expr("percentile_approx(price, 0.99)").alias("p99"),
    min("price").alias("min_price"),
    max("price").alias("max_price")
).collect()[0]

print("\n=== PRICE STATISTICS ===")
print(f"Mean: ${price_stats['mean_price']:,.2f}")
print(f"Std Dev: ${price_stats['std_price']:,.2f}")
print(f"Min: ${price_stats['min_price']:,.2f}")
print(f"Max: ${price_stats['max_price']:,.2f}")
print(f"1st percentile: ${price_stats['p01']:,.2f}")
print(f"99th percentile: ${price_stats['p99']:,.2f}")

# Define outlier thresholds
price_lower = max(500, price_stats['p01'])  # At least $500
price_upper = min(75000, price_stats['p99'])  # At most $75,000

print(f"\nOutlier thresholds: ${price_lower:,.2f} - ${price_upper:,.2f}")

In [ ]:
# Remove outliers and report
outliers_removed = ny_cars_cleaned.filter(
    (col("price") < price_lower) | (col("price") > price_upper)
)

ny_cars = ny_cars_cleaned.filter(
    (col("price") >= price_lower) & (col("price") <= price_upper)
)

print(f"\n=== OUTLIER REMOVAL REPORT ===")
print(f"Records before: {ny_cars_cleaned.count():,}")
print(f"Outliers removed: {outliers_removed.count():,} ({outliers_removed.count()/ny_cars_cleaned.count()*100:.2f}%)")
print(f"Records after: {ny_cars.count():,}")

# Show outlier distribution
print("\nOutlier breakdown:")
outliers_removed.select(
    count(when(col("price") < price_lower, 1)).alias("too_low"),
    count(when(col("price") > price_upper, 1)).alias("too_high")
).show()

In [ ]:
# Add derived columns for dimensional analysis
ny_cars_final = ny_cars \
    .withColumn("date", to_date(col("posting_date"))) \
    .withColumn("year_month", date_format(col("posting_date"), "yyyy-MM")) \
    .withColumn("year_week", date_format(col("posting_date"), "yyyy-ww")) \
    .withColumn("vehicle_age", year(col("posting_date")) - col("year")) \
    .withColumn(
        "age_category",
        when(col("vehicle_age") <= 3, "0-3 years")
        .when(col("vehicle_age") <= 7, "4-7 years")
        .when(col("vehicle_age") <= 12, "8-12 years")
        .otherwise("12+ years")
    ) \
    .withColumn(
        "vehicle_type_clean",
        coalesce(lower(trim(col("type"))), lit("unknown"))
    ) \
    .withColumn(
        "manufacturer_clean",
        coalesce(lower(trim(col("manufacturer"))), lit("unknown"))
    ) \
    .withColumn(
        "region_clean",
        coalesce(lower(trim(col("region"))), lit("unknown"))
    )

print("\nFinal car dataset prepared")
ny_cars_final.select(
    "date", "year_month", "price", "manufacturer_clean", 
    "vehicle_type_clean", "age_category", "region_clean"
).show(10)

### 3.2 Prepare Taxi Data (Filter 2019-2020, Clean, Remove Outliers)

In [ ]:
# Identify datetime columns (names vary across datasets)
taxi_columns = taxi_df.columns
pickup_col = [c for c in taxi_columns if 'pickup' in c.lower() and 'datetime' in c.lower()][0]
dropoff_col = [c for c in taxi_columns if 'dropoff' in c.lower() and 'datetime' in c.lower()][0]

print(f"Using columns: {pickup_col}, {dropoff_col}")

# Clean and filter taxi data
taxi_cleaned = taxi_df \
    .filter(col(pickup_col).isNotNull()) \
    .withColumn("pickup_datetime", to_timestamp(col(pickup_col))) \
    .withColumn("dropoff_datetime", to_timestamp(col(dropoff_col))) \
    .filter(year(col("pickup_datetime")).isin([2019, 2020])) \
    .filter(col("pickup_datetime") < col("dropoff_datetime"))

print(f"\nTaxi records in 2019-2020: {taxi_cleaned.count():,}")

In [ ]:
# Check for distance/location columns
distance_col = None
for col_name in ['trip_distance', 'distance']:
    if col_name in taxi_columns:
        distance_col = col_name
        break

if distance_col:
    # Remove trip distance outliers
    taxi_distance_stats = taxi_cleaned.select(
        expr(f"percentile_approx({distance_col}, 0.99)").alias("p99_distance"),
        max(distance_col).alias("max_distance")
    ).collect()[0]
    
    max_trip_distance = min(50, taxi_distance_stats['p99_distance'])  # Max 50 miles
    
    outliers_taxi = taxi_cleaned.filter(
        (col(distance_col) <= 0) | (col(distance_col) > max_trip_distance)
    )
    
    taxi_cleaned = taxi_cleaned.filter(
        (col(distance_col) > 0) & (col(distance_col) <= max_trip_distance)
    )
    
    print(f"\n=== TAXI OUTLIER REMOVAL ===")
    print(f"Outliers removed: {outliers_taxi.count():,}")
    print(f"Remaining records: {taxi_cleaned.count():,}")

In [ ]:
# Check for location columns (borough, zone, or coordinates)
location_cols = [c for c in taxi_columns if any(x in c.lower() for x in ['borough', 'zone', 'location', 'pu', 'do'])]
print(f"\nLocation columns found: {location_cols}")

# Show sample of location data
if location_cols:
    taxi_cleaned.select(location_cols[:5]).show(5)

In [ ]:
# Add time dimensions to taxi data
taxi_final = taxi_cleaned \
    .withColumn("pickup_date", to_date(col("pickup_datetime"))) \
    .withColumn("pickup_year_month", date_format(col("pickup_datetime"), "yyyy-MM")) \
    .withColumn("pickup_year_week", date_format(col("pickup_datetime"), "yyyy-ww")) \
    .withColumn("pickup_hour", hour(col("pickup_datetime"))) \
    .withColumn("dropoff_date", to_date(col("dropoff_datetime"))) \
    .withColumn("trip_duration_min", 
                (unix_timestamp(col("dropoff_datetime")) - unix_timestamp(col("pickup_datetime"))) / 60)

print("\nTaxi dataset prepared")
taxi_final.select(
    "pickup_date", "pickup_year_month", "pickup_hour", "trip_duration_min"
).show(10)

## 4. Build Dimensional Model

### 4.1 Taxi Demand Facts (Trip Density by Time and Location)

In [ ]:
# Determine location dimension from taxi data
# Check for borough or zone columns
location_dim = None
if 'PULocationID' in taxi_columns:
    location_dim = 'PULocationID'
elif 'pickup_location_id' in taxi_columns:
    location_dim = 'pickup_location_id'
elif 'borough' in taxi_columns:
    location_dim = 'borough'

print(f"Using location dimension: {location_dim}")

# Create pickup trip density (daily aggregation)
pickup_density_daily = taxi_final \
    .groupBy("pickup_date") \
    .agg(
        count("*").alias("pickup_trip_count"),
        avg("trip_duration_min").alias("avg_trip_duration")
    ) \
    .withColumn("trip_type", lit("pickup"))

# Create dropoff trip density (daily aggregation)
dropoff_density_daily = taxi_final \
    .groupBy("dropoff_date") \
    .agg(
        count("*").alias("dropoff_trip_count"),
    ) \
    .withColumnRenamed("dropoff_date", "date")

# Combine into unified daily fact table
taxi_demand_daily = pickup_density_daily \
    .withColumnRenamed("pickup_date", "date") \
    .join(dropoff_density_daily, "date", "outer") \
    .fillna(0) \
    .withColumn("total_trip_count", col("pickup_trip_count") + col("dropoff_trip_count")) \
    .withColumn("year_month", date_format(col("date"), "yyyy-MM")) \
    .withColumn("year_week", date_format(col("date"), "yyyy-ww"))

print(f"\nDaily taxi demand records: {taxi_demand_daily.count():,}")
taxi_demand_daily.orderBy("date").show(10)

In [ ]:
# Weekly aggregation
taxi_demand_weekly = taxi_demand_daily \
    .groupBy("year_week") \
    .agg(
        sum("pickup_trip_count").alias("pickup_trip_count"),
        sum("dropoff_trip_count").alias("dropoff_trip_count"),
        sum("total_trip_count").alias("total_trip_count"),
        avg("avg_trip_duration").alias("avg_trip_duration")
    )

print(f"\nWeekly taxi demand records: {taxi_demand_weekly.count():,}")
taxi_demand_weekly.orderBy("year_week").show(10)

In [ ]:
# Monthly aggregation
taxi_demand_monthly = taxi_demand_daily \
    .groupBy("year_month") \
    .agg(
        sum("pickup_trip_count").alias("pickup_trip_count"),
        sum("dropoff_trip_count").alias("dropoff_trip_count"),
        sum("total_trip_count").alias("total_trip_count"),
        avg("avg_trip_duration").alias("avg_trip_duration"),
        count("date").alias("days_in_period")
    ) \
    .withColumn("avg_daily_trips", col("total_trip_count") / col("days_in_period"))

print(f"\nMonthly taxi demand records: {taxi_demand_monthly.count():,}")
taxi_demand_monthly.orderBy("year_month").show()

### 4.2 Car Price Facts (Median Price by Time and Dimensions)

In [ ]:
# Daily car price aggregation (median)
car_price_daily = ny_cars_final \
    .groupBy("date") \
    .agg(
        expr("percentile_approx(price, 0.5)").alias("median_price"),
        avg("price").alias("avg_price"),
        count("*").alias("listing_count"),
        expr("percentile_approx(price, 0.25)").alias("p25_price"),
        expr("percentile_approx(price, 0.75)").alias("p75_price")
    ) \
    .withColumn("year_month", date_format(col("date"), "yyyy-MM")) \
    .withColumn("year_week", date_format(col("date"), "yyyy-ww"))

print(f"\nDaily car price records: {car_price_daily.count():,}")
car_price_daily.orderBy("date").show(10)

In [ ]:
# Weekly car price aggregation
car_price_weekly = ny_cars_final \
    .groupBy("year_week") \
    .agg(
        expr("percentile_approx(price, 0.5)").alias("median_price"),
        avg("price").alias("avg_price"),
        count("*").alias("listing_count")
    )

print(f"\nWeekly car price records: {car_price_weekly.count():,}")
car_price_weekly.orderBy("year_week").show(10)

In [ ]:
# Monthly car price aggregation
car_price_monthly = ny_cars_final \
    .groupBy("year_month") \
    .agg(
        expr("percentile_approx(price, 0.5)").alias("median_price"),
        avg("price").alias("avg_price"),
        count("*").alias("listing_count"),
        expr("percentile_approx(price, 0.25)").alias("p25_price"),
        expr("percentile_approx(price, 0.75)").alias("p75_price")
    )

print(f"\nMonthly car price records: {car_price_monthly.count():,}")
car_price_monthly.orderBy("year_month").show()

In [ ]:
# Car price by vehicle type dimension (monthly)
car_price_by_type_monthly = ny_cars_final \
    .groupBy("year_month", "vehicle_type_clean") \
    .agg(
        expr("percentile_approx(price, 0.5)").alias("median_price"),
        count("*").alias("listing_count")
    ) \
    .filter(col("listing_count") >= 10)  # Filter out sparse categories

print(f"\nCar price by type (monthly): {car_price_by_type_monthly.count():,}")
car_price_by_type_monthly.orderBy("year_month", desc("listing_count")).show(20)

In [ ]:
# Car price by age category (monthly)
car_price_by_age_monthly = ny_cars_final \
    .groupBy("year_month", "age_category") \
    .agg(
        expr("percentile_approx(price, 0.5)").alias("median_price"),
        count("*").alias("listing_count")
    ) \
    .filter(col("listing_count") >= 10)

print(f"\nCar price by age (monthly): {car_price_by_age_monthly.count():,}")
car_price_by_age_monthly.orderBy("year_month", "age_category").show(20)

### 4.3 Combined Dimensional Model (Join Facts)

In [ ]:
# Join daily facts
combined_daily = taxi_demand_daily \
    .join(car_price_daily, "date", "inner") \
    .select(
        "date",
        "year_month",
        "year_week",
        "pickup_trip_count",
        "dropoff_trip_count",
        "total_trip_count",
        "avg_trip_duration",
        "median_price",
        "avg_price",
        "listing_count",
        "p25_price",
        "p75_price"
    ) \
    .orderBy("date")

print(f"\nCombined daily records: {combined_daily.count():,}")
combined_daily.show(10)

In [ ]:
# Join weekly facts
combined_weekly = taxi_demand_weekly \
    .join(car_price_weekly, "year_week", "inner") \
    .orderBy("year_week")

print(f"\nCombined weekly records: {combined_weekly.count():,}")
combined_weekly.show(10)

In [ ]:
# Join monthly facts
combined_monthly = taxi_demand_monthly \
    .join(car_price_monthly, "year_month", "inner") \
    .select(
        "year_month",
        col("pickup_trip_count").alias("monthly_pickups"),
        col("dropoff_trip_count").alias("monthly_dropoffs"),
        col("total_trip_count").alias("monthly_total_trips"),
        col("avg_daily_trips"),
        col("avg_trip_duration"),
        col("median_price"),
        col("avg_price"),
        col("listing_count"),
        col("p25_price"),
        col("p75_price")
    ) \
    .orderBy("year_month")

print(f"\nCombined monthly records: {combined_monthly.count():,}")
combined_monthly.show()

## 5. Correlation Analysis

### 5.1 Overall Correlation (Time Series)

In [ ]:
# Calculate correlation at different time granularities
# Convert to Pandas for easier correlation calculation
combined_daily_pd = combined_daily.toPandas()
combined_weekly_pd = combined_weekly.toPandas()
combined_monthly_pd = combined_monthly.toPandas()

print("=== CORRELATION ANALYSIS ===")
print("\n1. Daily Granularity:")
print(f"   Pickup trips vs Median price: {combined_daily_pd['pickup_trip_count'].corr(combined_daily_pd['median_price']):.4f}")
print(f"   Dropoff trips vs Median price: {combined_daily_pd['dropoff_trip_count'].corr(combined_daily_pd['median_price']):.4f}")
print(f"   Total trips vs Median price: {combined_daily_pd['total_trip_count'].corr(combined_daily_pd['median_price']):.4f}")

print("\n2. Weekly Granularity:")
print(f"   Pickup trips vs Median price: {combined_weekly_pd['pickup_trip_count'].corr(combined_weekly_pd['median_price']):.4f}")
print(f"   Dropoff trips vs Median price: {combined_weekly_pd['dropoff_trip_count'].corr(combined_weekly_pd['median_price']):.4f}")
print(f"   Total trips vs Median price: {combined_weekly_pd['total_trip_count'].corr(combined_weekly_pd['median_price']):.4f}")

print("\n3. Monthly Granularity:")
print(f"   Monthly pickups vs Median price: {combined_monthly_pd['monthly_pickups'].corr(combined_monthly_pd['median_price']):.4f}")
print(f"   Monthly dropoffs vs Median price: {combined_monthly_pd['monthly_dropoffs'].corr(combined_monthly_pd['median_price']):.4f}")
print(f"   Monthly total trips vs Median price: {combined_monthly_pd['monthly_total_trips'].corr(combined_monthly_pd['median_price']):.4f}")
print(f"   Avg daily trips vs Median price: {combined_monthly_pd['avg_daily_trips'].corr(combined_monthly_pd['median_price']):.4f}")

In [ ]:
# Full correlation matrix (monthly)
print("\n=== MONTHLY CORRELATION MATRIX ===")
corr_matrix = combined_monthly_pd[[
    'monthly_pickups', 'monthly_dropoffs', 'monthly_total_trips',
    'avg_daily_trips', 'avg_trip_duration', 'median_price', 'avg_price'
]].corr()

print(corr_matrix)

### 5.2 Lagged Correlation (Check if taxi demand predicts future prices)

In [ ]:
# Compute lagged correlations (monthly)
combined_monthly_pd_sorted = combined_monthly_pd.sort_values('year_month')

print("\n=== LAGGED CORRELATION ANALYSIS ===")
print("(Does taxi demand predict future car prices?)\n")

for lag in [1, 2, 3]:
    combined_monthly_pd_sorted[f'median_price_lag_{lag}'] = combined_monthly_pd_sorted['median_price'].shift(lag)
    corr = combined_monthly_pd_sorted['monthly_total_trips'].corr(
        combined_monthly_pd_sorted[f'median_price_lag_{lag}']
    )
    print(f"Lag {lag} month: Trips vs Price: {corr:.4f}")

print("\n(Does car price predict future taxi demand?)\n")
for lag in [1, 2, 3]:
    combined_monthly_pd_sorted[f'trips_lag_{lag}'] = combined_monthly_pd_sorted['monthly_total_trips'].shift(lag)
    corr = combined_monthly_pd_sorted['median_price'].corr(
        combined_monthly_pd_sorted[f'trips_lag_{lag}']
    )
    print(f"Lag {lag} month: Price vs Trips: {corr:.4f}")

## 6. Summary Statistics

In [ ]:
print("=== DATASET SUMMARY ===")
print(f"\nTime period: 2019-2020")
print(f"\nTaxi trips analyzed: {taxi_final.count():,}")
print(f"NY car listings analyzed: {ny_cars_final.count():,}")
print(f"\nDaily observations: {combined_daily.count():,}")
print(f"Weekly observations: {combined_weekly.count():,}")
print(f"Monthly observations: {combined_monthly.count():,}")

print("\n=== KEY INSIGHTS ===")
print(f"Average daily taxi trips: {combined_monthly_pd['avg_daily_trips'].mean():,.0f}")
print(f"Average median car price: ${combined_monthly_pd['median_price'].mean():,.2f}")
print(f"Price range: ${combined_monthly_pd['median_price'].min():,.2f} - ${combined_monthly_pd['median_price'].max():,.2f}")
print(f"Trip volume range: {combined_monthly_pd['monthly_total_trips'].min():,.0f} - {combined_monthly_pd['monthly_total_trips'].max():,.0f}")

## 7. Export Combined Datasets for Visualization

In [ ]:
# Save combined datasets
combined_daily.write.mode('overwrite').parquet('data/combined_daily.parquet')
combined_weekly.write.mode('overwrite').parquet('data/combined_weekly.parquet')
combined_monthly.write.mode('overwrite').parquet('data/combined_monthly.parquet')

# Also save by vehicle type
car_price_by_type_monthly.write.mode('overwrite').parquet('data/price_by_type_monthly.parquet')
car_price_by_age_monthly.write.mode('overwrite').parquet('data/price_by_age_monthly.parquet')

print("Combined datasets saved to parquet files")

## 8. Suggested Visualizations

### Graph Recommendations to Test the Hypothesis:

#### 1. **Time Series Overlay Plot** (Primary)
   - **X-axis**: Date (monthly)
   - **Y-axis (left)**: Taxi trip count
   - **Y-axis (right)**: Median car price
   - **Purpose**: Visually inspect if peaks/troughs align
   - **Expected if hypothesis true**: Patterns should move together

#### 2. **Scatter Plot with Trend Line**
   - **X-axis**: Total taxi trips (monthly)
   - **Y-axis**: Median car price
   - **Color**: Time progression (gradient)
   - **Purpose**: Show direct relationship and correlation strength
   - **Expected if hypothesis true**: Positive correlation, upward trend

#### 3. **Pickup vs Dropoff Comparison**
   - **Type**: Dual scatter plots
   - **Plot 1**: Pickup trips vs Median price
   - **Plot 2**: Dropoff trips vs Median price
   - **Purpose**: Determine if pickup or dropoff is stronger predictor
   - **Expected if hypothesis true**: One should show stronger correlation

#### 4. **Correlation Heatmap**
   - **Variables**: All metrics (pickups, dropoffs, duration, price, etc.)
   - **Purpose**: Identify all relationships in the data
   - **Expected if hypothesis true**: Strong correlation values between demand and price

#### 5. **Lagged Correlation Plot**
   - **X-axis**: Lag period (0, 1, 2, 3 months)
   - **Y-axis**: Correlation coefficient
   - **Lines**: Trips→Price and Price→Trips
   - **Purpose**: Determine if there's a leading/lagging relationship
   - **Expected if hypothesis true**: Peak correlation at some lag

#### 6. **Vehicle Type Breakdown** (If plausible)
   - **Type**: Small multiples or faceted scatter plots
   - **Each panel**: Different vehicle type (sedan, SUV, truck, etc.)
   - **Purpose**: See if relationship varies by car type
   - **Expected if hypothesis true**: Some types show stronger correlation

#### 7. **Vehicle Age Breakdown**
   - **Type**: Faceted line charts
   - **Each panel**: Different age category
   - **Purpose**: Determine if newer/older cars react differently
   - **Expected if hypothesis true**: Certain age groups more sensitive

#### 8. **Seasonality Decomposition**
   - **Type**: Multiple time series showing trend, seasonal, residual
   - **Purpose**: Separate seasonal effects from true correlation
   - **Expected if hypothesis true**: Residuals should still correlate

#### 9. **Rolling Correlation**
   - **X-axis**: Date
   - **Y-axis**: 3-month rolling correlation
   - **Purpose**: Show if relationship strengthens/weakens over time
   - **Expected if hypothesis true**: Stable positive correlation

#### 10. **Box Plots by Quartiles**
   - **X-axis**: Taxi demand quartiles (low, med-low, med-high, high)
   - **Y-axis**: Car price distribution
   - **Purpose**: Show if high demand periods have higher prices
   - **Expected if hypothesis true**: Ascending median prices across quartiles

### Critical Analysis Notes:

**Potential Issues to Watch For:**
1. **Confounding variables**: Both metrics might be driven by external factors (economy, season)
2. **Spurious correlation**: Time trends might create false correlation
3. **Geographic mismatch**: Taxi data is NYC-specific, Craigslist is NY state-wide
4. **Sample bias**: Craigslist may not represent all car sales
5. **COVID-19 impact**: 2020 data includes pandemic effects on both metrics

**Graph Implementation Code Below:**

### 8.1 Graph 1: Time Series Overlay

In [ ]:
# Prepare data
combined_monthly_pd['year_month_dt'] = pd.to_datetime(combined_monthly_pd['year_month'])

# Create figure with secondary y-axis
fig = go.Figure()

# Add taxi trips trace
fig.add_trace(go.Scatter(
    x=combined_monthly_pd['year_month_dt'],
    y=combined_monthly_pd['monthly_total_trips'],
    name='Total Taxi Trips',
    line=dict(color='blue', width=2),
    yaxis='y'
))

# Add car price trace
fig.add_trace(go.Scatter(
    x=combined_monthly_pd['year_month_dt'],
    y=combined_monthly_pd['median_price'],
    name='Median Car Price',
    line=dict(color='red', width=2),
    yaxis='y2'
))

# Update layout
fig.update_layout(
    title='NYC Taxi Demand vs Used Car Prices Over Time (2019-2020)',
    xaxis=dict(title='Month'),
    yaxis=dict(
        title='Total Taxi Trips',
        titlefont=dict(color='blue'),
        tickfont=dict(color='blue')
    ),
    yaxis2=dict(
        title='Median Car Price ($)',
        titlefont=dict(color='red'),
        tickfont=dict(color='red'),
        overlaying='y',
        side='right'
    ),
    hovermode='x unified',
    height=500
)

fig.show()

print("\n📊 Graph 1: If hypothesis is true, you should see the two lines moving in similar patterns.")

### 8.2 Graph 2: Scatter Plot with Trend Line

In [ ]:
# Create scatter plot
fig = px.scatter(
    combined_monthly_pd,
    x='monthly_total_trips',
    y='median_price',
    color='year_month_dt',
    trendline='ols',
    labels={
        'monthly_total_trips': 'Total Monthly Taxi Trips',
        'median_price': 'Median Car Price ($)',
        'year_month_dt': 'Month'
    },
    title='Correlation: Taxi Trip Volume vs Car Prices',
    hover_data=['year_month']
)

fig.update_layout(height=500)
fig.show()

corr_val = combined_monthly_pd['monthly_total_trips'].corr(combined_monthly_pd['median_price'])
print(f"\n📊 Graph 2: Correlation coefficient = {corr_val:.4f}")
print("If hypothesis is true, expect positive correlation (upward trend line).")

### 8.3 Graph 3: Pickup vs Dropoff Comparison

In [ ]:
from plotly.subplots import make_subplots

# Create subplots
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Pickup Trips vs Price', 'Dropoff Trips vs Price')
)

# Pickup scatter
fig.add_trace(
    go.Scatter(
        x=combined_monthly_pd['monthly_pickups'],
        y=combined_monthly_pd['median_price'],
        mode='markers',
        marker=dict(color='blue', size=8),
        name='Pickups'
    ),
    row=1, col=1
)

# Dropoff scatter
fig.add_trace(
    go.Scatter(
        x=combined_monthly_pd['monthly_dropoffs'],
        y=combined_monthly_pd['median_price'],
        mode='markers',
        marker=dict(color='red', size=8),
        name='Dropoffs'
    ),
    row=1, col=2
)

fig.update_xaxes(title_text="Monthly Pickups", row=1, col=1)
fig.update_xaxes(title_text="Monthly Dropoffs", row=1, col=2)
fig.update_yaxes(title_text="Median Price ($)", row=1, col=1)
fig.update_yaxes(title_text="Median Price ($)", row=1, col=2)

fig.update_layout(
    title_text="Pickup vs Dropoff: Which Better Predicts Car Prices?",
    height=400
)

fig.show()

pickup_corr = combined_monthly_pd['monthly_pickups'].corr(combined_monthly_pd['median_price'])
dropoff_corr = combined_monthly_pd['monthly_dropoffs'].corr(combined_monthly_pd['median_price'])

print(f"\n📊 Graph 3:")
print(f"Pickup correlation: {pickup_corr:.4f}")
print(f"Dropoff correlation: {dropoff_corr:.4f}")
print(f"Stronger predictor: {'Pickups' if abs(pickup_corr) > abs(dropoff_corr) else 'Dropoffs'}")

### 8.4 Graph 4: Correlation Heatmap

In [ ]:
# Select numeric columns for correlation
corr_cols = [
    'monthly_pickups', 'monthly_dropoffs', 'monthly_total_trips',
    'avg_daily_trips', 'avg_trip_duration', 'median_price', 'avg_price'
]

corr_matrix = combined_monthly_pd[corr_cols].corr()

# Create heatmap
fig = go.Figure(data=go.Heatmap(
    z=corr_matrix.values,
    x=corr_matrix.columns,
    y=corr_matrix.columns,
    colorscale='RdBu',
    zmid=0,
    text=corr_matrix.values,
    texttemplate='%{text:.3f}',
    textfont={"size": 10}
))

fig.update_layout(
    title='Correlation Matrix: All Variables',
    height=600,
    xaxis={'side': 'bottom'}
)

fig.show()

print("\n📊 Graph 4: Look for strong correlations (close to +1 or -1) between trip metrics and price.")

### 8.5 Graph 5: Lagged Correlation Plot

In [ ]:
# Calculate lagged correlations
lags = list(range(0, 6))
trips_to_price_corr = []
price_to_trips_corr = []

combined_monthly_sorted = combined_monthly_pd.sort_values('year_month')

for lag in lags:
    if lag == 0:
        # No lag
        trips_to_price_corr.append(
            combined_monthly_sorted['monthly_total_trips'].corr(combined_monthly_sorted['median_price'])
        )
        price_to_trips_corr.append(
            combined_monthly_sorted['median_price'].corr(combined_monthly_sorted['monthly_total_trips'])
        )
    else:
        # Trips leading price
        trips_to_price_corr.append(
            combined_monthly_sorted['monthly_total_trips'].iloc[:-lag].corr(
                combined_monthly_sorted['median_price'].iloc[lag:]
            )
        )
        # Price leading trips
        price_to_trips_corr.append(
            combined_monthly_sorted['median_price'].iloc[:-lag].corr(
                combined_monthly_sorted['monthly_total_trips'].iloc[lag:]
            )
        )

# Plot
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=lags,
    y=trips_to_price_corr,
    mode='lines+markers',
    name='Trips → Price (trips lead)',
    line=dict(color='blue', width=2)
))

fig.add_trace(go.Scatter(
    x=lags,
    y=price_to_trips_corr,
    mode='lines+markers',
    name='Price → Trips (price leads)',
    line=dict(color='red', width=2)
))

fig.add_hline(y=0, line_dash="dash", line_color="gray")

fig.update_layout(
    title='Lagged Correlation: Does One Variable Predict the Other?',
    xaxis_title='Lag (months)',
    yaxis_title='Correlation Coefficient',
    height=500
)

fig.show()

print("\n📊 Graph 5: If one line peaks at lag > 0, that variable predicts the other.")
print("Example: Peak in 'Trips → Price' at lag 2 means trips predict prices 2 months later.")

### 8.6 Graph 6: Vehicle Type Breakdown

In [ ]:
# Load vehicle type data
price_by_type_pd = car_price_by_type_monthly.toPandas()

# Get top vehicle types by listing count
top_types = price_by_type_pd.groupby('vehicle_type_clean')['listing_count'].sum().nlargest(6).index
price_by_type_filtered = price_by_type_pd[price_by_type_pd['vehicle_type_clean'].isin(top_types)]

# Create faceted line chart
fig = px.line(
    price_by_type_filtered,
    x='year_month',
    y='median_price',
    color='vehicle_type_clean',
    facet_col='vehicle_type_clean',
    facet_col_wrap=3,
    title='Median Car Price Over Time by Vehicle Type',
    labels={'median_price': 'Median Price ($)', 'year_month': 'Month'},
    height=600
)

fig.update_xaxes(tickangle=45)
fig.show()

print("\n📊 Graph 6: Compare price trends across vehicle types. Do some types show stronger correlation with taxi demand?")
print("\nNote: To fully test this, you would need to overlay taxi demand on each panel.")

### 8.7 Graph 7: Vehicle Age Breakdown

In [ ]:
# Load vehicle age data
price_by_age_pd = car_price_by_age_monthly.toPandas()

# Create line chart
fig = px.line(
    price_by_age_pd,
    x='year_month',
    y='median_price',
    color='age_category',
    title='Median Car Price Over Time by Vehicle Age',
    labels={'median_price': 'Median Price ($)', 'year_month': 'Month'},
    height=500
)

fig.update_xaxes(tickangle=45)
fig.show()

print("\n📊 Graph 7: Do newer or older cars show more sensitivity to taxi demand changes?")

### 8.8 Graph 8: Box Plots by Demand Quartiles

In [ ]:
# Create demand quartiles
combined_monthly_pd['demand_quartile'] = pd.qcut(
    combined_monthly_pd['monthly_total_trips'],
    q=4,
    labels=['Q1 (Low)', 'Q2 (Med-Low)', 'Q3 (Med-High)', 'Q4 (High)']
)

# Create box plot
fig = px.box(
    combined_monthly_pd,
    x='demand_quartile',
    y='median_price',
    title='Car Price Distribution by Taxi Demand Quartile',
    labels={'demand_quartile': 'Taxi Demand Level', 'median_price': 'Median Car Price ($)'},
    height=500
)

fig.show()

print("\n📊 Graph 8: If hypothesis is true, median prices should increase across quartiles.")
print("\nMedian prices by quartile:")
print(combined_monthly_pd.groupby('demand_quartile')['median_price'].median())

## 9. Interpretation Guide

### How to Use These Graphs:

1. **Strong Evidence FOR hypothesis:**
   - Graph 1: Lines move together (same peaks/troughs)
   - Graph 2: Clear upward trend, correlation > 0.5
   - Graph 3: Both pickup/dropoff show positive correlation
   - Graph 4: Dark red cells between trip metrics and prices
   - Graph 5: Peak at lag 0, showing concurrent relationship
   - Graph 8: Ascending median prices across quartiles

2. **Evidence AGAINST hypothesis:**
   - Graph 1: Lines move independently or opposite directions
   - Graph 2: Flat or negative trend, correlation near 0
   - Graph 4: Weak correlations (near 0)
   - Graph 8: No clear pattern across quartiles

3. **Confounding Evidence (needs deeper investigation):**
   - Graph 1: Both trending same direction (could be macro trend, not causal)
   - Graph 5: Strong correlation at non-zero lag (indirect relationship)
   - Graphs 6-7: Some categories correlate, others don't (partial relationship)

### Statistical Considerations:

⚠️ **WARNING: Potential Flaws to Check:**

1. **Geographic Mismatch**: Taxi data is NYC-only, car data is NY state-wide. This is a fundamental issue that weakens the analysis.

2. **COVID-19 Impact**: 2020 data includes pandemic, which affected both metrics dramatically but independently.

3. **Spurious Correlation**: Both variables might trend together due to external factors (economy, season) without causal relationship.

4. **Sample Bias**: Craigslist may not represent the full used car market.

5. **Time Aggregation**: Monthly aggregation might hide daily/weekly patterns.

### Recommendations:

- **Focus on pre-COVID months** (2019 and Jan-Feb 2020) for cleaner analysis
- **Consider controlling for seasonality** before claiming causation
- **Look for alternative explanations** if correlation exists
- **Check if relationship makes economic sense**: Why would taxi demand affect car prices?

### Economic Logic Test:

**Possible mechanisms IF correlation exists:**
- High taxi demand → People need transportation → Higher car prices (demand pressure)
- High taxi demand → Economic activity indicator → More disposable income → Higher car prices

**Counter-arguments:**
- More taxi availability → Less need to own cars → Lower car prices (opposite)
- Both driven by separate economic factors (population, employment)

**Verdict**: Use these graphs to test the hypothesis, but be skeptical and look for confounding variables!

In [ ]:
print("\n" + "="*80)
print("ANALYSIS COMPLETE")
print("="*80)
print("\nNext steps:")
print("1. Review all graphs above")
print("2. Look for consistent patterns across multiple visualizations")
print("3. Consider alternative explanations for any correlations found")
print("4. Focus on pre-COVID data (2019) for cleaner signal")
print("5. Be aware of geographic mismatch (NYC taxi vs NY state cars)")
print("\nRemember: Correlation ≠ Causation")